<div style="padding: 30px 60px; border: 5px skyblue solid; border-radius: 30px; max-width: 600px; font-size: 2rem; line-height: 3rem; color: black; font-family: sans-serif;">

  <div style="font-size: 1.5rem; font-weight: 200;">[CM3070] Final Project - BSc CS University of London</div>
  <div style="font-size: 1.5rem; font-weight: 500;">Deep Learning on Satellite Imagery</div>
  <div style="font-size: 2rem; color: dodgerblue; font-weight: bold;">by Arjun Bajaj</div>
</div>

# Converting to TensorFlow Lite for Deployment on the Pico

This notebook focuses on converting the best (final) model created in the previous notebook, into the TensorFlow Lite format to deploy it on the Raspberry Pi Pico Microcontroller.

To convert a Keras model into the `TFLite` format, the model has to be loaded, the converter has to be initialized, and optimizations configured. The converter also needs a function which provides it with sample data so that the model is configured appropriately for production. This _Representative Dataset Generator_ ensures that the Tensor Arenas are of the correct types and sizes.

The `TFLite` model is written to disk, but it cannot be used directly on the Pico, as most microcontrollers (including the Pico) do not have a filesystem to load files from. Hence, the `TFLite` binary is converted into a `C Header` file which allows the model to be embedded into the `C++` code directly.

When converting the model from `Keras` to `TFLite` some optimizations are applied such as quantization, which reduce the model size considerably. However, these optimizations come at a slight cost of accuracy. To ensure the model is performing well, the accuracy can be tested in Python. This is done below.

In [11]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import subprocess
import shutil
import pathlib

import tensorflow as tf
import numpy as np
import sklearn.metrics as metrics

from core import load_eurosat

In [3]:
# Load the validation and test datasets.
# The validation dataset is used to check the accuracy difference
# between the Keras and TFLite models. The test dataset is only used
# for the Representative Dataset Generator at this stage.
(_, val, test) = load_eurosat()

Found 21600 files belonging to 10 classes.
Found 2700 files belonging to 10 classes.
Found 2700 files belonging to 10 classes.


In [6]:
# Load the final model (trained in the previous notebook)
model = tf.keras.models.load_model("../artifacts/eurosat.keras")

# Intialize the TFLite converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply Default Optimizations
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Create the Representative Dataset Generator
def representative_dataset_generator():
  # Take the first batch of images
  images, _labels = test.take(1).as_numpy_iterator().next()

  # Convert the images to float32 with a surrounding dimension
  for image in images:
    yield [np.array(image, dtype=np.float32, ndmin=4)]

# Set the generator function on the converter's object
converter.representative_dataset = representative_dataset_generator

# Invoke the converter to generate the TFLite model
tflite_model = converter.convert()

# Write the TFLite model to disk
open("../artifacts/eurosat.tflite", "wb").write(tflite_model)

# Convert the TFLite file to a C Header file.
# This requires the `xxd` command, usually available on Linux and macOS.
subprocess.run(["xxd", "-i", "eurosat.tflite", "eurosat.h"], cwd="../artifacts")

# Move the C Header file to the Pico's source directory
shutil.move("../artifacts/eurosat.h", "../lite/pico/eurosat.h")

print('Model Converted')

INFO:tensorflow:Assets written to: /var/folders/2z/5ll18j2j6xx6m8q0zjbjpj480000gn/T/tmp0e75jegl/assets


INFO:tensorflow:Assets written to: /var/folders/2z/5ll18j2j6xx6m8q0zjbjpj480000gn/T/tmp0e75jegl/assets
fully_quantize: 0, inference_type: 6, input_inference_type: 0, output_inference_type: 0


Model Converted


In [7]:
# Print the size of the TFLite file and the converted Header File
tflite_size = pathlib.Path('../artifacts/eurosat.tflite').stat().st_size / 1024
header_size = pathlib.Path('../lite/pico/eurosat.h').stat().st_size / 1024
print(f"TFLite Model Size: {tflite_size:.2f} kb")
print(f"Header File Size: {header_size:.2f} kb")

TFLite Model Size: 36.41 kb
Header File Size: 224.63 kb


# Evaluating Accuracy of TFLite Model

In this step, the validation accuracy of the TFLite model is evaluated, as converting the model leads to a slight drop in accuracy.

In [15]:
# Load the TFLite Model
lite = tf.lite.Interpreter('../artifacts/eurosat.tflite')

# Allocate Tensors
lite.allocate_tensors()

# Variables to store the total and correct predictions
true = []
pred = []

# Iterate over the validation dataset
for image, label in val.unbatch():
  # Write the image to the input tensor
  lite.set_tensor(lite.get_input_details()[0]['index'], [image])

  # Invoke the TFLite model inference
  lite.invoke()

  # Read the result from the output tensor
  output = lite.get_tensor(lite.get_output_details()[0]['index'])
  
  # Get the predicted label using the argmax function
  result = np.argmax(output)

  # Append the true and predicted labels to the lists
  true.append(label)
  pred.append(result)


val_accuracy = metrics.accuracy_score(true, pred)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Validation Accuracy: 90.96%


# Results

The final model's highest validation accuracy was `91.33%`. After converting the `Keras` model to `TFLite`, the validation accuracy is `90.96%`, leading to a drop of `0.37` percentage points.

In the next notebook, the final model is evaluated on the test dataset, and the inference results from the Pico are explored and visualized.

----